In [1]:
import motmetrics as mm  # 导入该库
import numpy as np

使用这个库的流程就是，先导入两个txt文件（分别是GT和自己生成的结果，且都是标准的MOTChallenge格式，具体格式在MOTChallenge网站上有详细介绍），再创建accumulator，然后逐帧将每帧的GT和自己跟踪的结果（用hypotheses表示）填充至该accumulator中，然后使用度量器计算即可得到字符串summary，里面就是计算的结果，再将summary打印出来即可。

In [4]:
metrics = list(mm.metrics.motchallenge_metrics)  # # 即支持的所有metrics的名字列表
print(metrics)

['idf1', 'idp', 'idr', 'recall', 'precision', 'num_unique_objects', 'mostly_tracked', 'partially_tracked', 'mostly_lost', 'num_false_positives', 'num_misses', 'num_switches', 'num_fragmentations', 'mota', 'motp', 'num_transfer', 'num_ascend', 'num_migrate']


In [5]:
acc = mm.MOTAccumulator(auto_id=True) # 创建accumulator
# 用第一帧填充该accumulator
acc.update(
    [1, 2],                     # Ground truth objects in this frame
    [1, 2, 3],                  # Detector hypotheses in this frame
    [
        [0.1, np.nan, 0.3],     # Distances from object 1 to hypotheses 1, 2, 3
        [0.5,  0.2,   0.3]      # Distances from object 2 to hypotheses 1, 2, 3
    ]
)

# 查看该帧的事件
print(acc.events) # a pandas DataFrame containing all events

                Type  OId  HId    D
FrameId Event                      
0       0        RAW  NaN  NaN  NaN
        1        RAW  1.0  1.0  0.1
        2        RAW  1.0  3.0  0.3
        3        RAW  2.0  1.0  0.5
        4        RAW  2.0  2.0  0.2
        5        RAW  2.0  3.0  0.3
        6      MATCH  1.0  1.0  0.1
        7      MATCH  2.0  2.0  0.2
        8         FP  NaN  3.0  NaN


In [6]:
# 只查看MOT事件，不查看RAW
print(acc.mot_events) # a pandas DataFrame containing MOT only events

                Type  OId  HId    D
FrameId Event                      
0       6      MATCH  1.0  1.0  0.1
        7      MATCH  2.0  2.0  0.2
        8         FP  NaN  3.0  NaN


In [7]:
# 继续填充下一帧
frameid = acc.update(
    [1, 2],  # GT
    [1],     # hypotheses
    [
        [0.2],
        [0.4]
    ]
)
print(acc.mot_events.loc[frameid])

        Type  OId  HId    D
Event                      
3      MATCH  1.0  1.0  0.2
4       MISS  2.0  NaN  NaN


导入txt文件来创建accumulator并填充


In [ ]:
gt_file="/path/gt.txt"
"""  文件格式如下
1,0,1255,50,71,119,1,1,1
2,0,1254,51,71,119,1,1,1
3,0,1253,52,71,119,1,1,1
...
"""

ts_file="/path/test.txt"
"""  文件格式如下
1,1,1240.0,40.0,120.0,96.0,0.999998,-1,-1,-1
2,1,1237.0,43.0,119.0,96.0,0.999998,-1,-1,-1
3,1,1237.0,44.0,117.0,95.0,0.999998,-1,-1,-1
...
"""

gt = mm.io.loadtxt(gt_file, fmt="mot15-2D", min_confidence=1)  # 读入GT
ts = mm.io.loadtxt(ts_file, fmt="mot15-2D")  # 读入自己生成的跟踪结果

acc=mm.utils.compare_to_groundtruth(gt, ts, 'iou', distth=0.5)  # 根据GT和自己的结果，生成accumulator，distth是距离阈值

mh = mm.metrics.create()

# 打印单个accumulator
summary = mh.compute(acc,
                     metrics=['num_frames', 'mota', 'motp'], # 一个list，里面装的是想打印的一些度量
                     name='acc') # 起个名
print(summary)

# 打印多个accumulators
summary = mh.compute_many([acc, acc.events.loc[0:1]], # 多个accumulators组成的list
                          metrics=['num_frames', 'mota', 'motp'], 
                          name=['full', 'part']) # 起个名
print(summary)

# 自定义显示格式
strsummary = mm.io.render_summary(
    summary,
    formatters={'mota' : '{:.2%}'.format},  # 将MOTA的格式改为百分数显示
    namemap={'mota': 'MOTA', 'motp' : 'MOTP'}  # 将列名改为大写
)
print(strsummary)


# mh模块中有内置的显示格式
summary = mh.compute_many([acc, acc.events.loc[0:1]],
                          metrics=mm.metrics.motchallenge_metrics,
                          names=['full', 'part'])

strsummary = mm.io.render_summary(
    summary,
    formatters=mh.formatters,
    namemap=mm.io.motchallenge_metric_names
)
print(strsummary)